In [24]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

import numpy as np
pio.renderers.default = 'notebook'

In [25]:
#df = pd.read_excel("SISMOGRAMAS CBART.xlsx")
#df.to_csv("SISMOGRAMAS CBART.csv", index=False)

In [26]:
#df = pd.read_excel("BASE_SISMOS.xlsx")
df = pd.read_csv("BASE_SISMOS.csv")
df_simsg = pd.read_csv("SISMOGRAMAS CBART.csv")

In [27]:
def plot_anim(axis_x, axis_y, axis_z, interval):
    # Create figure
    fig = go.Figure(go.Scatter3d(x=[], y=[], z=[],
                                mode="markers",
                                marker=dict(color="red", size=1)
                                )
                    )

        
    # Frames
    frames = [go.Frame(data= [go.Scatter3d(
                            x=axis_x[:k+1],
                            y=axis_y[:k+1],
                            z=axis_z[:k+1])
                            ],
                    traces= [0],
                    name=f'frame{k}'      
                    )for k in range(0, len(axis_x)-1, interval) #for k in range(len(aux)-1)
            ]

    fig.update(frames=frames)




    def frame_args(duration):
        return {
                "frame": {"duration": duration},
                "mode": "immediate",
                "fromcurrent": True,
                "transition": {"duration": duration, "easing": "linear"},
                }


    sliders = [
        {"pad": {"b": 10, "t": 60},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        
        "steps": [
                    {"args": [[f.name], frame_args(0)],
                    "label": str(k),
                    "method": "animate",
                    } for k, f in enumerate(fig.frames)
                ]
        }
            ]

    fig.update_layout(

        updatemenus = [{"buttons":[
                        {
                            "args": [None, frame_args(50)],
                            "label": "Play", 
                            "method": "animate",
                        },
                        {
                            "args": [[None], frame_args(0)],
                            "label": "Pause", 
                            "method": "animate",
                    }],
                        
                    "direction": "left",
                    "pad": {"r": 10, "t": 70},
                    "type": "buttons",
                    "x": 0.1,
                    "y": 0,
                }
            ],
            sliders=sliders,
            width=900,
            height=800
        )

    fig.update_layout(scene = dict(xaxis=dict(range=[min(axis_x), max(axis_x)], autorange=False),
                                yaxis=dict(range=[min(axis_y), max(axis_y)], autorange=False),
                                zaxis=dict(range=[min(axis_z), max(axis_z)], autorange=False)
                                )
                    )
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 0.1
    fig.update_layout(sliders=sliders)
    fig.show()

In [28]:
def menu():
    i = 1

    ## seleccionar sismo
    sismos = list(df.SISMO.unique())
    print('SELECCIONAR SISMO:')
    for s in sismos:
        print(str(i)+') '+s)
        i+=1
    print('\n')
    
    ops_sism = int()
    while not(any(np.isin(range(len(sismos)), int(ops_sism)-1))):
        ops_sism = input('')

    print(sismos[int(ops_sism)-1])

    ## seleccionar estación
    estaciones = list(df[df.SISMO == sismos[int(ops_sism)-1]].ESTACION.unique())
    i = 1
    print('SELECCIONAR ESTACIÓN:')
    for s in estaciones:
        print(str(i)+') '+s)
        i+=1
    print('\n')

    ops_est = int()
    while not(any(np.isin(range(len(estaciones)), int(ops_est)-1))):
        ops_est = input('')

    print(estaciones[int(ops_est)-1])
    print('\n')
    print('SISMO : '+sismos[int(ops_sism)-1], 'ESTACIÓN : '+estaciones[int(ops_est)-1], sep='\n')
    min = df[(df.SISMO == sismos[int(ops_sism)-1])&(df.ESTACION == estaciones[int(ops_est)-1])]['TIEMPO'].min()
    max = df[(df.SISMO == sismos[int(ops_sism)-1])&(df.ESTACION == estaciones[int(ops_est)-1])]['TIEMPO'].max()
    print('mínimo : '+str(min), 'máximo : '+str(max), sep='\n')

    return sismos[int(ops_sism)-1], estaciones[int(ops_est)-1]

In [33]:
def menu_simsg():
    #print('QUE MUESTRA DE BETULIA?')
    #print('7 u 8 :')

    op = ''
    while op not in ['7', '8']:
        op = input('QUE MUESTRA DE BETULIA? 7 u 8 :')

    if op == '7':
        return('BETULIA 7')
    else:
        return('BETULIA 8')

In [44]:
rango_y = [df_simsg['ACELERACION'].min(), df_simsg['ACELERACION'].max()]
rango_x = [df_simsg['TIEMPO'].min(), df_simsg['TIEMPO'].max()]

## MENÚ

In [30]:
sismo, estacion = menu()

SELECCIONAR SISMO:
1) BETULIA
2) LOS SANTOS
3) ORTEGA
4) NIDO B.MANGA
5) QUETAME


BETULIA
SELECCIONAR ESTACIÓN:
1) CBART 8
2) CCORP 3
3) CEING 4
4) CFLOD 2
5) CGRAL
6) CUAGR
7) CUAGR DH
8) CUSAL 3
9) CTVCA 2
10) CPSUB
11) CAVIA 2
12) CAVIA 3
13) CTUNA 1
14) CTUNA 2
15) CFONT 1
16) CFONT 2
17) CRADI 1
18) CRADI 2
19) CRADI 3


CTUNA 1


SISMO : BETULIA
ESTACIÓN : CTUNA 1
mínimo : 0.0
máximo : 20.975


## GRAFICAR ANIMACIÓN

In [45]:
minimo = 10
maximo = 30
x = df[(df.SISMO == sismo) & (df.ESTACION == estacion)&(df.TIEMPO >= minimo)&(df.TIEMPO <= maximo)]['EW (X)']
y = df[(df.SISMO == sismo) & (df.ESTACION == estacion)&(df.TIEMPO >= minimo)&(df.TIEMPO <= maximo)]['NS (Y)']
z = df[(df.SISMO == sismo) & (df.ESTACION == estacion)&(df.TIEMPO >= minimo)&(df.TIEMPO <= maximo)]['VER (Z)']

plot_anim(x, y, z, 100)

#SISMOGRAMA
if sismo == 'BETULIA':
    sismo_gram = menu_simsg()
else:
    sismo_gram = sismo
fig = px.line(df_simsg[df_simsg.SISMO == sismo_gram], x='TIEMPO', y='ACELERACION', title=sismo_gram,
                range_x=rango_x, range_y=rango_y)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [43]:
rango_y

[0.0, 179.995]